### Required downloads:

In [ ]:
%pip install pandas
%pip install together

### Import sample recipes:

In [1]:
import pandas as pd
recipe_df = pd.read_csv('data_recipe_unnumbered.csv')
print("recipe_df content: \n", recipe_df)

recipe_df content: 
                                        title_en  \
0                  fried beef with spring onion   
1    Whitebait and Green Onion Omelet Triangles   
2           Braised Chicken Wings with Potatoes   
3          Simple Hong Kong Style Curry Chicken   
4          Hong Kong Style Steamed Pork and Egg   
5  Hong Kong Style Scallop Egg White Fried Rice   
6            Salted Fish and Chicken Fried Rice   
7     Stir-Fried Water Spinach with Beef Slices   
8           Homemade Braised Chicken Drumsticks   
9                   Stir-Fried Beef with Onions   

                                       ingredient_en  \
0  Beef, green onion, garlic, ginger, oyster sauc...   
1  Eggs, green onions, dried whitebait, salt, bla...   
2  Chicken wings, potatoes, onion, dark soy sauce...   
3  Spring chicken, onion, carrot, potato, broccol...   
4  Eggs, water, minced meat, soy sauce, salt, gre...   
5   Cold rice, Dried scallops, Choy sum stems, Eg...   
6  Chicken fillet, salted

In [2]:
# Testing: Find rows that contain 'eggs' in the ingredient_en column
food_preference = ["eggs", "green onions"]
matching_recipes = recipe_df[recipe_df['ingredient_en'].str.contains(f', {food_preference[0]},', case=False, na=False)]
print(matching_recipes)

                             title_en  \
6  Salted Fish and Chicken Fried Rice   

                                       ingredient_en  \
6  Chicken fillet, salted fish, rice, water, eggs...   

                                      instruction_en title_zh  \
6  1\nPrepare cold rice (2 cups of rice, 2 cups o...   鹹魚雞粒炒飯   

                  ingredient_zh  \
6  雞柳, 鹹魚, 米, 水, 雞蛋, 蘿蔔, 西芹, 洋蔥   

                                      instruction_zh portion  \
6  1\n先備冷飯 （2杯米，2杯水）\n\n2\n雞柳切粒，醃15分鐘以上。蘿蔔、西芹、洋蔥及...       3   

                                         source_link  
6  https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/1721...  


# AI Conversation setup

In [3]:
import os
from together import Together
os.environ["TOGETHER_API_KEY"] = "f00e4f3943a505ce68ff471fb2976c33040fcbb6c16874121ad76dbc53b70952" 
# toaster's together key
import json
import re

ingredient_list = [
    'Beef', 'Green onion', 'Garlic', 'Ginger', 'Oyster sauce', 'Sugar', 'Cornstarch', 'Oil', 'Salt', 'Soy sauce', 
    'Black pepper powder', 'Eggs', 'Dried whitebait', 'Chicken wings', 'Potatoes', 'Onion', 'Dark soy sauce', 
    'Light soy sauce', 'Pepper', 'Sesame oil', 'Spring chicken', 'Carrot', 'Broccoli', 'Monk fruit sugar', 
    'Rice wine', 'Curry powder', 'Evaporated milk', 'Minced meat', 'Cold rice', 'Dried scallops', 
    'Choy sum stems', 'Egg whites', 'Coriander', 'Seasoning', 'Rice wine', 'Chicken fillet', 'Salted fish', 
    'Rice', 'Water spinach', 'Beef slices', 'Minced garlic', 'Chicken powder', 'Chili', 'Chicken drumsticks', 
    'Star anise', 'Cotton tofu', 'Braising sauce', 'Palm sugar or brown sugar'
]

In [25]:
# stream = client.chat.completions.create(
#   model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
#   messages=[{"role": "user", "content": "What are some fun things to do in New York?"}],
#   stream=True,
# )

# for chunk in stream:
#   print(chunk.choices[0].delta.content or "", end="", flush=True)

In [41]:
together = Together(api_key=os.environ.get("TOGETHER_API_KEY"))

with open("functions.json", "r") as json_file:
    funcs = json.load(json_file)

funcs[0]["function"]["parameters"]["properties"]["include"]["items"]["enum"] = ingredient_list
# funcs[0]["function"]["parameters"]["properties"]["exclude"]["items"]["enum"] = ingredient_list

### Prompt built-in method:

In [ ]:
# Has yet to attempt getting this method to work

# toolPrompt = f"""
# You have access to the following functions:

# {"\n\n".join([(f"Use the function '{func["name"]}' to '{func["description"]}':\n" 
#                f"{json.dumps(func)}") for func in funcs])}

# If you choose to call a function ONLY reply in the following format with no prefix or suffix:

# <function=example_function_name>{{\"example_name\": \"example_value\"}}</function>

# Reminder:
# - Function calls MUST follow the specified format, start with <function= and end with </function>
# - Required parameters MUST be specified
# - Only call one function at a time
# - Put the entire function call reply on one line
# - If there is no function call available, answer the question like normal with your current knowledge and do not tell the user about function calls

# """

# messages = [
#   	{
#         "role": "system",
#         "content": toolPrompt,
#     },
#     {
#         "role": "user",
#         "content": "What is the weather in Tokyo?",
#     },
    
# ]

# response = together.chat.completions.create(
#     model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
#     messages=messages,
#     max_tokens=1024,
#     temperature=0,
# )

messages.append(response.choices[0].message)
print(response.choices[0].message.content)

In [ ]:
# def parse_tool_response(response: str):
#     function_regex = r"<function=(\w+)>(.*?)</function>"
#     match = re.search(function_regex, response)

#     if match:
#         function_name, args_string = match.groups()
#         try:
#             args = json.loads(args_string)
#             return {
#                 "function": function_name,
#                 "arguments": args,
#             }
#         except json.JSONDecodeError as error:
#             print(f"Error parsing function arguments: {error}")
#             return None
#     return None

# parsed_response = parse_tool_response(response.choices[0].message.content)
# print(parse_tool_response(response.choices[0].message.content))

### Tool parameter method: 

In [49]:
user_question = "Can you recommend me what to cook? If possible, please include ginger and potatoes. Exclude apples"

messages = [
    {"role": "system", 
     "content": 
     "You are a helpful assistant that can access external functions. "
     "The responses from these function calls will be appended to this dialogue. "
     "Please provide responses based on the information from these function calls."},
    {"role": "user", 
     "content": user_question}
]
    
# Notable issue: When user asks to exclude an item not in our built in list of ingredients, AI panics and excludes everything that isn't included. More testing required.

response = together.chat.completions.create(
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
    messages=messages,
    tools=funcs,
    tool_choice="auto",
)

print(json.dumps(response.choices[0].message.model_dump()['tool_calls'], indent=2))

[
  {
    "id": "call_xmsaf35y4fu80v9ec5zzjpb9",
    "type": "function",
    "function": {
      "name": "recommend_recipes",
      "arguments": "{\"include\":[\"Ginger\",\"Potatoes\"],\"exclude\":[\"Apples\"],\"headcount\":3}"
    }
  }
]


### Key function: recommend_recipes()

In [50]:
recipe_df = pd.read_csv('data_recipe_unnumbered.csv')

def recommend_recipes(include: list[str], exclude: list[str], headcount: int=1):
    global recipe_df
    print("Include: ", include)
    print("Exclude: ", exclude)
    print("Headcount: ", headcount)
    remaining_recipes = recipe_df
    # first we remove recipes containing excluded ingredients
    for ingredient in exclude:
        # print(~remaining_recipes['ingredient_en'].str.contains(', '+ingredient+',', case=False, na=False))
        remaining_recipes = remaining_recipes[~remaining_recipes['ingredient_en'].str.contains(', '+ingredient+',', case=False, na=False)]
        print("remaining_recipes: \n", 
              remaining_recipes[~remaining_recipes['ingredient_en'].str.contains(', '+ingredient+',', case=False, na=False)])
    include_set = set([i.lower() for i in include])
    ans = []
    # Naive implementation: repeatedly pick the recipe that ticks off the most ingredients in the list
    for i in range(max(headcount-1, 1)):
        print(f'Dish {i}:') 
        remaining_recipes['matches'] = remaining_recipes['ingredient_en'].apply(lambda x: len(set(x.split(', ')).intersection(set(include_set))))
        remaining_recipes = remaining_recipes.sort_values(by="matches", ascending=False)
        print("remaining_recipes: \n", remaining_recipes)
        head = remaining_recipes.head(1)
        print("Head: ", head)
        ans.append(head.T[head.index[0]].to_list())
        print(ans[-1])
        include_set = include_set - set([ing.lower() for ing in ans[-1][1].split(', ')])
    return_recipes = ""
    for i in range(len(ans)):
        return_recipes += f"Recipe {i+1}: {ans[i][0]}\nIngredients: {ans[i][1]}\nInstructions: {ans[i][2]}\n\n"
    return return_recipes

In [45]:
messages

[{'role': 'system',
  'content': 'You are a helpful assistant that can access external functions. The responses from these function calls will be appended to this dialogue. Please provide responses based on the information from these function calls.'},
 {'role': 'user',
  'content': 'Can you recommend me what to cook? If possible, please include ginger and potatoes. Exclude apples'},
 {'tool_call_id': 'call_dfg60nl321dgki62rhyl2a98',
  'role': 'tool',
  'name': 'recommend_recipes',
  'content': [['fried beef with spring onion',
    'Beef, green onion, garlic, ginger, oyster sauce, sugar, cornstarch, oil, salt, soy sauce, black pepper powder',
    ' 1. Marinate the beef. 2. Chop the ginger and garlic.\n\nWash and cut the green onions.\nSauté the ginger and green onions until fragrant.\nQuickly stir-fry the beef.\nOnce it starts to change color, add the green onions and stir-fry.\nAdd the prepared sauce and cook until done.',
    '蔥炒牛肉',
    '牛肉, 大蔥, 蒜頭, 薑 , 蠔油, 糖, 生粉, 油, 鹽, 生抽, 黑胡椒粉',
 

In [51]:
tool_calls = response.choices[0].message.tool_calls
if tool_calls:
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)

        if function_name == "recommend_recipes":
            function_response = recommend_recipes(
                include=function_args.get("include"),
                exclude=function_args.get("exclude"),
                headcount=function_args.get("headcount"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )
    function_enriched_response = together.chat.completions.create(
        model="mistralai/Mixtral-8x7B-Instruct-v0.1",
        messages=messages,
    )
    print(json.dumps(function_enriched_response.choices[0].message.model_dump(), indent=2)))


Include:  ['Ginger', 'Potatoes']
Exclude:  ['Apples']
Headcount:  3
remaining_recipes: 
                                        title_en  \
0                  fried beef with spring onion   
1    Whitebait and Green Onion Omelet Triangles   
2           Braised Chicken Wings with Potatoes   
3          Simple Hong Kong Style Curry Chicken   
4          Hong Kong Style Steamed Pork and Egg   
5  Hong Kong Style Scallop Egg White Fried Rice   
6            Salted Fish and Chicken Fried Rice   
7     Stir-Fried Water Spinach with Beef Slices   
8           Homemade Braised Chicken Drumsticks   
9                   Stir-Fried Beef with Onions   

                                       ingredient_en  \
0  Beef, green onion, garlic, ginger, oyster sauc...   
1  Eggs, green onions, dried whitebait, salt, bla...   
2  Chicken wings, potatoes, onion, dark soy sauce...   
3  Spring chicken, onion, carrot, potato, broccol...   
4  Eggs, water, minced meat, soy sauce, salt, gre...   
5   Cold rice

In [53]:
print(function_response)

Recipe 1: fried beef with spring onion
Ingredients: Beef, green onion, garlic, ginger, oyster sauce, sugar, cornstarch, oil, salt, soy sauce, black pepper powder
Instructions:  1. Marinate the beef. 2. Chop the ginger and garlic.

Wash and cut the green onions.
Sauté the ginger and green onions until fragrant.
Quickly stir-fry the beef.
Once it starts to change color, add the green onions and stir-fry.
Add the prepared sauce and cook until done.

Recipe 2: Braised Chicken Wings with Potatoes
Ingredients: Chicken wings, potatoes, onion, dark soy sauce, light soy sauce, pepper, sugar, sesame oil
Instructions: 1 Thaw the chicken wings, rinse, and drain. Mix with marinade and let sit for about half an hour.

2 Slice the onion.

3 Cut the potatoes into chunks.

4 Heat a pan over medium heat, add oil, and fry the chicken skin side down. Flip and fry until both sides are colored, then set aside.

5 Heat a pan over medium heat, add oil, and sauté ginger slices and onion. Add potatoes and fry u

In [154]:
recommend_recipes(['potatOES', 'ginGEr', 'RICE'], [], 3)

Dish 0:
['fried beef with spring onion', 'Beef, green onion, garlic, ginger, oyster sauce, sugar, cornstarch, oil, salt, soy sauce, black pepper powder', ' 1. Marinate the beef. 2. Chop the ginger and garlic.\n\nWash and cut the green onions.\nSauté the ginger and green onions until fragrant.\nQuickly stir-fry the beef.\nOnce it starts to change color, add the green onions and stir-fry.\nAdd the prepared sauce and cook until done.', '蔥炒牛肉', '牛肉, 大蔥, 蒜頭, 薑 , 蠔油, 糖, 生粉, 油, 鹽, 生抽, 黑胡椒粉', '牛肉醃好，薑蒜切好，大蔥洗乾淨並切斷。爆香薑蔥，將牛肉落鑊快炒。開始變色後加入大蔥翻炒，再加入調好的醬汁，煮熟即可。', '2', 'https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/16187234-%E6%B8%AF%E5%BC%8F%E5%AE%B6%E5%B8%B8%E8%8F%9C%E8%94%A5%E7%82%92%E7%89%9B%E8%82%89', 1]
Dish 1:
['Braised Chicken Wings with Potatoes', 'Chicken wings, potatoes, onion, dark soy sauce, light soy sauce, pepper, sugar, sesame oil', '1 Thaw the chicken wings, rinse, and drain. Mix with marinade and let sit for about half an hour.\n\n2 Slice the onion.\n\n3 Cut the potatoes into chunks.\n\n4 H

[['fried beef with spring onion',
  'Beef, green onion, garlic, ginger, oyster sauce, sugar, cornstarch, oil, salt, soy sauce, black pepper powder',
  ' 1. Marinate the beef. 2. Chop the ginger and garlic.\n\nWash and cut the green onions.\nSauté the ginger and green onions until fragrant.\nQuickly stir-fry the beef.\nOnce it starts to change color, add the green onions and stir-fry.\nAdd the prepared sauce and cook until done.',
  '蔥炒牛肉',
  '牛肉, 大蔥, 蒜頭, 薑 , 蠔油, 糖, 生粉, 油, 鹽, 生抽, 黑胡椒粉',
  '牛肉醃好，薑蒜切好，大蔥洗乾淨並切斷。爆香薑蔥，將牛肉落鑊快炒。開始變色後加入大蔥翻炒，再加入調好的醬汁，煮熟即可。',
  '2',
  'https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/16187234-%E6%B8%AF%E5%BC%8F%E5%AE%B6%E5%B8%B8%E8%8F%9C%E8%94%A5%E7%82%92%E7%89%9B%E8%82%89',
  1],
 ['Braised Chicken Wings with Potatoes',
  'Chicken wings, potatoes, onion, dark soy sauce, light soy sauce, pepper, sugar, sesame oil',
  '1 Thaw the chicken wings, rinse, and drain. Mix with marinade and let sit for about half an hour.\n\n2 Slice the onion.\n\n3 Cut the potatoes into chunks.

In [115]:
%pip install numpy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [116]:
import numpy as np

In [122]:
df = pd.DataFrame(np.arange(12).reshape(3, 4),
                  columns=['A', 'B', 'C', 'D'])
df = df.drop([0, 1])
print(df)

   A  B   C   D
2  8  9  10  11
